To train this agent, click _Runtime_ and press _Run all_. Make sure you've enabled a free Tesla T4 GPU!

<div class="align-center">
<a href="https://github.com/openpipe/art"><img src="https://github.com/openpipe/art/raw/main/assets/ART_pill.png" height="50"></a>
<a href="https://discord.gg/zbBHRUpwf4"><img src="https://github.com/openpipe/art/raw/main/assets/Discord_pill.png" height="50"></a>
<a href="https://art.openpipe.ai"><img src="https://github.com/openpipe/art/raw/main/assets/Documentation_pill.png" height="50"></a>

Questions? Join the Discord and ask away! For feature requests or to leave a star, visit our [GitHub](https://github.com/openpipe/art).

</div>

<a href="https://art.openpipe.ai/"><img src="https://github.com/openpipe/art/raw/main/assets/Header_separator.png" height="5"></a>

This notebook shows how to train a Qwen 2.5 3B model to play tic tac toe. It will demonstrate how to set up a multi-turn agent, how to train it, and how to evaluate it.

Completions and metrics will be logged to Weights & Biases.

You will learn how to construct an [agentic environment](#Environment), how to define a [rollout](#Rollout), and how to run a [training loop](#Loop).


In [1]:
!uv pip install "numpy<2.0.0"

Using Python 3.11.13 environment at: /usr
Resolved 1 package in 141ms
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠹ Preparing packages... (0/1)
⠹ Preparing packages... (0/1)
⠹ Preparing packages... (0/1)
⠹ Preparing packages... (0/1)
Prepared 1 package in 374ms
Uninstalled 1 package in 46ms
Installed 1 package in 20ms
 - numpy==2.0.2
 + numpy==1.26.4


### WARNING:

If you are running in Google Colab and installing numpy does not say "Requirement already satisfied: numpy<2.0.0" then click "Runtime" and "Restart Session."


In [1]:
# make sure we're using numpy 1.*.*
import numpy as np

if (np.__version__).startswith("1."):
    print("Numpy version is 1.*.*, you're good to go!")
else:
    raise ValueError("Please restart your runtime using the above instructions!")

Numpy version is 1.*.*, you're good to go!


### Environment Variables

Later on in the notebook, we'll be creating a model that can automatically logs metrics and chat completions to Weights & Biases. In order to do so, you'll need to provide your Weights & Biases API key as an environment variable.


In [2]:
import os
from google.colab import userdata

WANDB_API_KEY = None #userdata.get('wandb_token')
if WANDB_API_KEY:
    os.environ["WANDB_API_KEY"] = WANDB_API_KEY

### Installation


In [3]:
%%capture
!uv pip install openpipe-art==0.3.11.post1 accelerate==1.7.0 setproctitle weave --prerelease allow --no-cache-dir

### Agentic Environment

<a name="Environment"></a>

ART allows your agent to learn by interacting with its environment. In this example, we'll create an environment in which the agent can play tic tac toe.

Feel free to read as much or as little of this section's code as you'd like. The important thing to understand is that we're defining the rules of this agent's environment. In many cases, this will already be defined by the task you're trying to solve, but if you need to define a custom environment, this is how you do it.


In [4]:
import random
from typing import TypedDict
from typing import Literal
import xml.etree.ElementTree as ET


class TicTacToeGame(TypedDict):
    board: list[list[str]]
    agent_symbol: Literal["x", "o"]
    opponent_symbol: Literal["x", "o"]


def generate_game(board_length: int = 3) -> TicTacToeGame:
    board = [["_" for _ in range(board_length)] for _ in range(board_length)]
    agent_symbol = random.choice(["x", "o"])
    opponent_symbol = "x" if agent_symbol == "o" else "o"
    return {
        "board": board,
        "agent_symbol": agent_symbol,
        "opponent_symbol": opponent_symbol,
    }


def render_board(game: TicTacToeGame) -> str:
    board = game["board"]
    board_length = len(board)
    # print something like this:
    #    1   2   3
    # A  _ | x | x
    # B  o | _ | _
    # C  _ | o | _
    # where _ is an empty cell

    board_str = "   " + "   ".join([str(i + 1) for i in range(board_length)]) + "\n"
    for i in range(board_length):
        board_str += f"{chr(65 + i)}  {board[i][0]} | {board[i][1]} | {board[i][2]}\n"
    return board_str


def get_opponent_move(game: TicTacToeGame) -> tuple[int, int]:
    # get a random empty cell
    empty_cells = [
        (i, j) for i in range(3) for j in range(3) if game["board"][i][j] == "_"
    ]
    return random.choice(empty_cells)


def apply_agent_move(game: TicTacToeGame, move: str) -> None:
    board_length = len(game["board"])

    try:
        root = ET.fromstring(move)
        square = root.text
    except Exception:
        raise ValueError("Invalid xml")

    try:
        row_index = ord(square[0]) - 65
        col_index = int(square[1]) - 1
    except Exception as e:
        print(e)
        raise ValueError("Unable to parse square")

    if (
        row_index < 0
        or row_index >= board_length
        or col_index < 0
        or col_index >= board_length
    ):
        raise ValueError(
            f"Invalid move, row or column out of bounds: {row_index}, {col_index}"
        )

    # check if the move is valid
    if game["board"][row_index][col_index] != "_":
        raise ValueError("Square already occupied")

    game["board"][row_index][col_index] = game["agent_symbol"]


def check_winner(board: list[list[str]]) -> Literal["x", "o", "draw", None]:
    board_length = len(board)
    # check rows
    for row in board:
        if row.count(row[0]) == board_length and row[0] != "_":
            return row[0]
    # check columns
    for col in range(board_length):
        if [board[row][col] for row in range(board_length)].count(
            board[0][col]
        ) == board_length and board[0][col] != "_":
            return board[0][col]

    # top right to bottom left
    upward_diagonal = [board[i][board_length - i - 1] for i in range(board_length)]
    if (
        upward_diagonal.count(upward_diagonal[0]) == board_length
        and upward_diagonal[0] != "_"
    ):
        return upward_diagonal[0]

    # top left to bottom right
    downward_diagonal = [board[i][i] for i in range(board_length)]
    if (
        downward_diagonal.count(downward_diagonal[0]) == board_length
        and downward_diagonal[0] != "_"
    ):
        return downward_diagonal[0]

    # check for draw
    if all(cell != "_" for row in board for cell in row):
        return "draw"
    return None

### Creating a Model

Now that we've defined the rules of our environment, we can create a model that will learn to play 2048. We'll use a Qwen 2.5 3B model for this example. The `name` parameter will be associated with a wandb run, and the `base_model` parameter is the model that we'll be training a LoRA on top of.


In [5]:
import art
from dotenv import load_dotenv

from art.local import LocalBackend

load_dotenv()

random.seed(42)

backend = LocalBackend(path="./.art")

INFO 07-11 01:29:57 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 07-11 01:29:57 [__init__.py:239] Automatically detected platform cuda.


### Creating a Model

Now that we've defined the rules of our environment, we can create a model that will learn to play tic tac toe. We'll use a Qwen 2.5 3B model for this example. The `name` parameter will be associated with a wandb run, and the `base_model` parameter is the model that we'll be training a LoRA on top of.


In [6]:
import os

model = art.TrainableModel(
    name="ART_tictactoe_FT",
    project="tic-tac-toe",
    base_model="Qwen/Qwen2.5-3B-Instruct",
)
await model.register(backend)

### Defining a Rollout

<a name="Rollout"></a>

A rollout is a single episode of an agent performing its task. It generates one or more trajectories, which are lists of messages and choices.

In this example, the rollout function generates a game of tic tac toe, and the agent plays it until the game is finished. It then returns a trajectory which contains all the `system` and `user` messages presented to the agent, as well as all the `choices` that the agent made.

When the game is finished the `reward` for the agent's performance is calculated based on whether the agent won, lost, drew, or errored, which is then assigned to the trajectory.

This rollout function will be called many times in parallel during each step of the training loop.


In [7]:
import art
import openai
import math
from pydantic import BaseModel
import weave

if os.getenv("WANDB_API_KEY", ""):
    print("initializing weave")
    weave.init(model.project, settings={"print_call_link": False})


class TicTacToeScenario(BaseModel):
    step: int


@weave.op
@art.retry(exceptions=(openai.LengthFinishReasonError,))
async def rollout(model: art.Model, scenario: TicTacToeScenario) -> art.Trajectory:
    game = generate_game() #generates typed dict which is an instance of TicTacToeGame which itself has only 3 keys (board, agent symbol, opp symbol)

    trajectory = art.Trajectory(
        messages_and_choices=[
            {
                "role": "system",
                "content": f"You are a tic-tac-toe player. You are playing against an opponent. Always choose the move most likely to lead to an eventual win. Return your move as an XML object with a single property 'move', like so: <move>A1</move>. Optional moves are 'A1', 'B3', 'C2', etc. You are the {game['agent_symbol']} symbol.",
            }
        ],
        metadata={
            "notebook-id": "tic-tac-toe",
            "step": scenario.step,
        },
        reward=0,
    )

    move_number = 0

    if game["agent_symbol"] == "o":
        starting_opponent_move = get_opponent_move(game) #randomly selects and returns the empty cell to fill
        game["board"][starting_opponent_move[0]][starting_opponent_move[1]] = game[
            "opponent_symbol"
        ]

    while check_winner(game["board"]) is None:
        trajectory.messages_and_choices.append(
            {"role": "user", "content": render_board(game)} #adds the current state of the game to the messages and choices list
        )

        messages = trajectory.messages()

        try:
            client = model.openai_client()
            chat_completion = await client.chat.completions.create(
                model=model.get_inference_name(),
                messages=messages,
                max_completion_tokens=128,
            ) #pass state of game and previous messages to model
        except openai.LengthFinishReasonError as e:
            raise e
        except Exception as e:
            print("caught exception generating chat completion")
            print(e)
            global failing_trajectory
            failing_trajectory = trajectory
            raise e

        choice = chat_completion.choices[0] #get selected action
        content = choice.message.content
        assert isinstance(content, str)
        trajectory.messages_and_choices.append(choice) #add action to messages and chocies list

        try:
            apply_agent_move(game, content) #applies the agent's action (if not valid action, then a ValueError exception is thrown)
        except ValueError:
            #if an agent's move is invalid, then the reward for the episode/trajectory is  - 1 + epsilon where epsilon is positive if more than 100 moves were made ()
            trajectory.reward = -1 + (math.log(move_number + 1) / math.log(100))
            break

        move_number += 1
        if check_winner(game["board"]) is not None:
            break

        opponent_move = get_opponent_move(game)
        game["board"][opponent_move[0]][opponent_move[1]] = game["opponent_symbol"]

    winner = check_winner(game["board"])

    if winner == game["agent_symbol"]:
        trajectory.reward = 1
        trajectory.metrics["win"] = 1
    elif winner == game["opponent_symbol"]:
        trajectory.reward = 0
        trajectory.metrics["win"] = 0
    elif winner == "draw":
        trajectory.reward = 0.5
        trajectory.metrics["win"] = 0.5

    trajectory.metrics["num_moves"] = move_number

    return trajectory

  if event.key is 'enter':



<a name="Loop"></a>

### Training Loop

The training loop is where the magic happens. For each of the 100 steps defined below, the rollout function will be called 200 times in parallel. This means that 200 games will be played at once. Each game will produce a trajectory, which will be used to update the model.

The `gather` step will wait for all of the trajectories to be generated, then it will delete all but the most recent checkpoint and train the model on the new trajectories.

Inference will be blocked until the training is complete.


In [8]:
for i in range(await model.get_step(), 50):
    train_groups = await art.gather_trajectory_groups(
        (
            art.TrajectoryGroup(
                rollout(model, TicTacToeScenario(step=i)) for _ in range(48)
            )
            for _ in range(1)
        ),
        pbar_desc="gather",
    )
    await model.delete_checkpoints()
    await model.train(train_groups, config=art.TrainConfig(learning_rate=5e-5))

gather:   0%|          | 0/48 [00:00<?, ?it/s]

 (subsequent messages of this type will be suppressed)
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Deleted checkpoint ./.art/tic-tac-toe/models/ART_tictactoe_FT/0047
Packed 48 trajectories into 5 sequences of length 2048


train:   0%|          | 0/5 [00:00<?, ?it/s]

gather:   0%|          | 0/48 [00:00<?, ?it/s]

Deleted checkpoint ./.art/tic-tac-toe/models/ART_tictactoe_FT/0048
Packed 48 trajectories into 5 sequences of length 2048


train:   0%|          | 0/5 [00:00<?, ?it/s]

### Using the Model

Just like that, you've trained an agent to play tic tac toe! Now it's time to use your model outside of ART, in the wild! The easiest way to do that is to load it from disk, where it was saved after each training step, and either run inference on it locally or upload it to a central hub like HuggingFace.

Check out the code below for small demo of the model you just trained playing tic tac toe!


In [9]:
import torch
from unsloth import FastLanguageModel


# example: .art/tic-tac-toe/models/002/0003
lora_model_path = (
    f".art/{model.project}/models/{model.name}/{await model.get_step():04d}"
)

print(f"loading model from {lora_model_path}\n")

peft_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=lora_model_path,
    max_seq_length=16384,
    dtype=torch.bfloat16,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(peft_model)

game = generate_game()
move_number = 0

messages = [
    {
        "role": "system",
        "content": f"You are a tic-tac-toe player. You are playing against an opponent. Always choose the move most likely to lead to an eventual win. Return your move as an XML object with a single property 'move', like so: <move>A1</move>. Optional moves are 'A1', 'B3', 'C2', etc. You are the {game['agent_symbol']} symbol.",
    },
]

if game["agent_symbol"] == "o":
    starting_opponent_move = get_opponent_move(game)
    game["board"][starting_opponent_move[0]][starting_opponent_move[1]] = game[
        "opponent_symbol"
    ]

while check_winner(game["board"]) is None:
    rendered_board = render_board(game)
    messages.append({"role": "user", "content": rendered_board})

    inputs = tokenizer.apply_chat_template(
        messages, return_tensors="pt", add_generation_prompt=True
    ).to("cuda")

    content = ""

    def get_completion() -> str:
        with torch.no_grad():
            outputs = peft_model.generate(
                input_ids=inputs,
                max_new_tokens=100,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
            )
            return tokenizer.decode(
                outputs[0][inputs.shape[1] :], skip_special_tokens=True
            )

    try:
        content = get_completion()
    except Exception as e:
        print("caught exception generating chat completion", e)
        raise e

    messages.append({"role": "assistant", "content": content})

    try:
        apply_agent_move(game, content)
        move_number += 1
    except ValueError:
        raise ValueError(f"Invalid move on move {move_number}: {content}")

    # print the board every move
    print(f"\nmove {move_number}")
    print(f"board:\n{rendered_board}")
    print(f"agent move: {content}")
    print(f"updated board:\n{render_board(game)}")

    if check_winner(game["board"]) is not None:
        break
    move_number += 1

    opponent_move = get_opponent_move(game)
    game["board"][opponent_move[0]][opponent_move[1]] = game["opponent_symbol"]

winner = check_winner(game["board"])

print(f"game finished in {move_number} moves")

if winner == game["agent_symbol"]:
    print("game won! 💪")
elif winner == game["opponent_symbol"]:
    print("game lost! 😢")
elif winner == "draw":
    print("draw! 🤷‍♂️")


print(f"final board:\n\n{render_board(game)}")


Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
loading model from .art/tic-tac-toe/models/ART_tictactoe_FT/0050

==((====))==  Unsloth 2025.5.1: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Device does not support bfloat16. Will change to float16.
Unsloth 2025.5.1 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



move 1
board:
   1   2   3
A  _ | _ | _
B  _ | _ | _
C  _ | _ | _

agent move: <move>B3</move>
updated board:
   1   2   3
A  _ | _ | _
B  _ | _ | x
C  _ | _ | _


move 3
board:
   1   2   3
A  _ | o | _
B  _ | _ | x
C  _ | _ | _

agent move: <move>A3</move>
updated board:
   1   2   3
A  _ | o | x
B  _ | _ | x
C  _ | _ | _


move 5
board:
   1   2   3
A  _ | o | x
B  o | _ | x
C  _ | _ | _

agent move: <move>C3</move>
updated board:
   1   2   3
A  _ | o | x
B  o | _ | x
C  _ | _ | x

game finished in 5 moves
game won! 💪
final board:

   1   2   3
A  _ | o | x
B  o | _ | x
C  _ | _ | x



In [17]:

game = generate_game()
move_number = 0

messages = [
    {
        "role": "system",
        "content": f"You are a tic-tac-toe player. You are playing against an opponent. Always choose the move most likely to lead to an eventual win. Return your move as an XML object with a single property 'move', like so: <move>A1</move>. Optional moves are 'A1', 'B3', 'C2', etc. You are the {game['agent_symbol']} symbol.",
    },
]

if game["agent_symbol"] == "o":
    starting_opponent_move = get_opponent_move(game)
    game["board"][starting_opponent_move[0]][starting_opponent_move[1]] = game[
        "opponent_symbol"
    ]

while check_winner(game["board"]) is None:
    rendered_board = render_board(game)
    messages.append({"role": "user", "content": rendered_board})

    inputs = tokenizer.apply_chat_template(
        messages, return_tensors="pt", add_generation_prompt=True
    ).to("cuda")

    content = ""

    def get_completion() -> str:
        with torch.no_grad():
            outputs = peft_model.generate(
                input_ids=inputs,
                max_new_tokens=100,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
            )
            return tokenizer.decode(
                outputs[0][inputs.shape[1] :], skip_special_tokens=True
            )

    try:
        content = get_completion()
    except Exception as e:
        print("caught exception generating chat completion", e)
        raise e

    messages.append({"role": "assistant", "content": content})

    try:
        apply_agent_move(game, content)
        move_number += 1
    except ValueError:
        raise ValueError(f"Invalid move on move {move_number}: {content}")

    # print the board every move
    print(f"\nmove {move_number}")
    print(f"board:\n{rendered_board}")
    print(f"agent move: {content}")
    print(f"updated board:\n{render_board(game)}")

    if check_winner(game["board"]) is not None:
        break
    move_number += 1

    opponent_move = get_opponent_move(game)
    game["board"][opponent_move[0]][opponent_move[1]] = game["opponent_symbol"]

winner = check_winner(game["board"])

print(f"game finished in {move_number} moves")

if winner == game["agent_symbol"]:
    print("game won! 💪")
elif winner == game["opponent_symbol"]:
    print("game lost! 😢")
elif winner == "draw":
    print("draw! 🤷‍♂️")


print(f"final board:\n\n{render_board(game)}")


move 1
board:
   1   2   3
A  _ | _ | _
B  _ | _ | _
C  _ | _ | _

agent move: <move>B3</move>
updated board:
   1   2   3
A  _ | _ | _
B  _ | _ | x
C  _ | _ | _


move 3
board:
   1   2   3
A  _ | _ | _
B  _ | _ | x
C  _ | _ | o

agent move: <move>A3</move>
updated board:
   1   2   3
A  _ | _ | x
B  _ | _ | x
C  _ | _ | o



ValueError: Invalid move on move 4: The previous board state is already a win for you (by placing an 'x' in position B3, which completes a row of three 'x's horizontally). Given that you've already won, there's no need to make any more moves unless you're trying to force a win or prevent a loss, but since you've already won, there's no further action needed. If you want me to simulate a scenario where I'm trying to ensure a win, let me know and we can play

<div class="align-center">
<a href="https://github.com/openpipe/art"><img src="https://github.com/openpipe/art/raw/notebooks/assets/ART_pill.png" height="50"></a>
<a href="https://discord.gg/zbBHRUpwf4"><img src="https://github.com/openpipe/art/raw/notebooks/assets/Discord_pill.png" height="50"></a>
<a href="https://openpipe.ai/blog/art-e-mail-agent"><img src="https://github.com/openpipe/art/raw/main/assets/ART_E_pill.png" height="50"></a>

Questions? Join the Discord and ask away! For feature requests or to leave a star, visit our [Github](https://github.com/openpipe/art).

</div>
